In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

 98% 1.05G/1.06G [00:11<00:00, 146MB/s]
100% 1.06G/1.06G [00:11<00:00, 97.9MB/s]


In [3]:
import zipfile
zip_ref=zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [19]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras.applications.vgg16 import VGG16

In [20]:
conv_base=VGG16(include_top= False,
                weights= "imagenet",
                input_shape=(150,150,3) )

In [21]:
conv_base.trainable=True

set_trainable=False

for layer in conv_base.layers:
  if layer.name=='block5_conv1':
    set_trainable=True

  if set_trainable:
    layer.trainable=True
  else:
    layer.trainable=False


for layer in conv_base.layers:
  print(layer.name,layer.trainable)

input_2 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [7]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [22]:
model=Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [23]:
conv_base.trainable=False
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array,array_to_img,load_img

In [24]:
batch_size=32
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2,horizontal_flip=True )
test_datagen=ImageDataGenerator(rescale=1./255)

train_ds= keras.utils.image_dataset_from_directory(directory='/content/train', labels = "inferred",
                                                   label_mode = "int",
                                                   batch_size = 32,
                                                   image_size = (150,150))

validation_ds= keras.utils.image_dataset_from_directory(directory='/content/test', labels = "inferred",
                                                   label_mode = "int",
                                                   batch_size = 32,
                                                   image_size = (150,150))

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [25]:
# Normalization

def process(image,label):
  image=tf.cast(image/255.0,tf.float32)
  return image,label




In [26]:
train_ds=train_ds.map(process)
validation_ds=validation_ds.map(process)

In [27]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [28]:
history=model.fit_generator(train_ds,validation_data=validation_ds,epochs=10)

Epoch 1/10


<ipython-input-28-8c8d8ea3abbd>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_ds,validation_data=validation_ds,epochs=10)


625/625 [==============================] - 67s 104ms/step - loss: 0.2712 - accuracy: 0.8834 - val_loss: 0.2867 - val_accuracy: 0.8708
Epoch 2/10
625/625 [==============================] - 65s 103ms/step - loss: 0.1923 - accuracy: 0.9208 - val_loss: 0.2015 - val_accuracy: 0.9156
Epoch 3/10
625/625 [==============================] - 65s 103ms/step - loss: 0.1530 - accuracy: 0.9378 - val_loss: 0.2328 - val_accuracy: 0.9036
Epoch 4/10
625/625 [==============================] - 55s 88ms/step - loss: 0.1271 - accuracy: 0.9491 - val_loss: 0.2454 - val_accuracy: 0.9054
Epoch 5/10
625/625 [==============================] - 54s 86ms/step - loss: 0.1079 - accuracy: 0.9559 - val_loss: 0.2492 - val_accuracy: 0.9106
Epoch 6/10
625/625 [==============================] - 64s 102ms/step - loss: 0.0762 - accuracy: 0.9712 - val_loss: 0.2669 - val_accuracy: 0.9126
Epoch 7/10
625/625 [==============================] - 55s 88ms/step - loss: 0.0568 - accuracy: 0.9790 - val_loss: 0.3342 - val_accuracy: 0.9018